In [1]:
# ex2.ipynb

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# 讀取資料
data = pd.read_csv("https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv")
data = data.drop(['Time'], axis=1)
data['Amount'] = StandardScaler().fit_transform(data['Amount'].values.reshape(-1, 1))

X = data.drop(['Class'], axis=1)
y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 加入 Isolation Forest 得分為新特徵
iso = IsolationForest(contamination=0.002, random_state=42)
iso.fit(X_train)
train_scores = iso.decision_function(X_train)
test_scores = iso.decision_function(X_test)

X_train['iso_score'] = train_scores
X_test['iso_score'] = test_scores

# 使用 SMOTE 平衡資料
sm = SMOTE(random_state=42)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

# 用 XGBoost 訓練模型
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_res, y_train_res)

y_pred_xgb = xgb_model.predict(X_test)

print("=== Fusion Model (Isolation + XGBoost) Evaluation ===")
print(classification_report(y_test, y_pred_xgb))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [13:05:06] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


=== Fusion Model (Isolation + XGBoost) Evaluation ===
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.73      0.82      0.77       148

    accuracy                           1.00     85443
   macro avg       0.87      0.91      0.89     85443
weighted avg       1.00      1.00      1.00     85443

